## Master driver script for Tasks 1 - 3

In [1]:
# Master imports
# Train critic
from hw1 import Critic, Composer
from midi2seq import process_midi_seq, seq2piano, random_piano, piano2seq, segment

import torch # Requried to create tensors to store all numerical values
import torch.nn as nn # Required for weight and bias tensors
import torch.nn.functional as F # Required for the activation functions
from torch.utils.data import TensorDataset, DataLoader, Dataset # How do we use them?

import numpy as np
root = './'
midi_sav_dir = './midi_test' #! This folder must be created manually before executing this script
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Global variables
batch_size = 50
seq_to_gen = 10000
seq_len = 51 # Hardcoded, actual lenght is 50, +1 to accomodate training composer
epoch = 10

midi_load = process_midi_seq(datadir=root, n=seq_to_gen, maxlen=(seq_len - 1)) # Master dataset to be used by both Composer and Critic
midi_load_bck = np.copy(midi_load)

INFO:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:NumExpr defaulting to 8 threads.


### Task 2 Train Composer

In [2]:
# Train Composer
conv_val = 0.001
conv_flag = False
loss = None
loader_composer = DataLoader(TensorDataset(torch.from_numpy(midi_load)), shuffle=True, batch_size=batch_size, num_workers=4) # Composer training dataset

cps_train = Composer(seq_len = (seq_len - 1)) # Loads composer model with default values
for i in range(epoch):
    for idx, x in enumerate(loader_composer):
        loss = cps_train.train(x[0].cuda(0).long())
        print(f"Epoch: {i}, Batch: {idx}, Loss: {loss.item()}")
        if (loss.item() < conv_val):
            print(f"convergence reached")
            conv_flag = True
        
        if(conv_flag):
            break # 
    
    if(conv_flag):
            print(f"Training complete with loss: {loss.item()}")
            break # 

##Save model
#* Activate to save model
# torch.save(cps.model.state_dict(), "az_cps.pth")

ComposerLSTM(
  (embedding): Embedding(382, 10)
  (lstm): LSTM(10, 64, num_layers=2, batch_first=True, dropout=0.3)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_1): Linear(in_features=50, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=382, bias=True)
  (softmax): Softmax(dim=1)
)
Loss in this bach: 5.927250862121582
Epoch: 0, Batch: 0, Loss: 5.927250862121582
Loss in this bach: 5.854522228240967
Epoch: 0, Batch: 1, Loss: 5.854522228240967
Loss in this bach: 5.691117763519287
Epoch: 0, Batch: 2, Loss: 5.691117763519287
Loss in this bach: 5.424749374389648
Epoch: 0, Batch: 3, Loss: 5.424749374389648
Loss in this bach: 4.817628383636475
Epoch: 0, Batch: 4, Loss: 4.817628383636475
Loss in this bach: 4.704041481018066
Epoch: 0, Batch: 5, Loss: 4.704041481018066
Loss in this bach: 5.205347061157227
Epoch: 0, Batch: 6, Loss: 5.205347061157227
Loss in this bach: 4.699106216430664
Epoch: 0, Batch: 7, Loss: 4.6

In [3]:
# Test composition of music
# cps_test = Composer(load_trained=True)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Will have access to this from hw1.py
# print()

# midi = cps_test.compose(n = 200)
# midi = seq2piano(midi)
# midi.write("midi_test.midi")

### Task 1: Train Critic

In [4]:
# Train Critic
loss = None
conv_flag = False
conv_val = 0.001

midi_load = np.copy(midi_load_bck)
# midi_load = process_midi_seq(datadir=root, n=seq_to_gen, maxlen=(seq_len - 1)) # Numpy array
# #print(f"midi_load shape: {midi_load.shape}")

# Split into training and testing dataset
#* Adopted from https://github.com/alizano94/CSC-7343-HW1/blob/main/HW1/main.py
#* Modified to conform to shape requirements

train_percent = 0.8
train_size = int(train_percent*midi_load.shape[0])

train_seq_data = midi_load[0:(int(train_size)), :-1]
test_seq_data = midi_load[(int(train_size)):,:-1]

train_labels = np.ones(train_seq_data.shape[0])
test_labels = np.ones(test_seq_data.shape[0])

# Create bad samples for training
for i in range(train_seq_data.shape[0]):
    rand_midi = random_piano(n = 250)
    rand_seq = piano2seq(rand_midi)
    rand_seq =  rand_seq[0:seq_len - 1]
    train_seq_data = np.vstack((train_seq_data, rand_seq))
    train_labels = np.append(train_labels,0)

# Create bad samples for testing
for i in range(test_seq_data.shape[0]):
    rand_midi = random_piano(n = 250)
    rand_seq = piano2seq(rand_midi)
    rand_seq =  rand_seq[0:seq_len - 1]
    test_seq_data = np.vstack((test_seq_data, rand_seq))
    test_labels = np.append(test_labels,0)

x_train_tensors = torch.Tensor(train_seq_data)
y_train_tensors = torch.Tensor(train_labels)

x_test_tensors = torch.Tensor(test_seq_data)
y_test_tensors = torch.Tensor(test_labels)

#* Debug
print(x_train_tensors.shape)
# print(y_train_tensors.shape)
print(x_test_tensors.shape)
# print(y_test_tensors.shape)
# print(x_train_tensors_final)

dataset = TensorDataset(x_train_tensors, y_train_tensors)
test_dataset = TensorDataset(x_test_tensors, y_test_tensors)

loader_critic = DataLoader(dataset, batch_size=batch_size, shuffle=True) # Critic training
loader_critic_tester = DataLoader(test_dataset, batch_size=batch_size, shuffle=True) # One sequence at a time


torch.Size([16686, 50])
torch.Size([4172, 50])


In [5]:
# Train Critic
ctr_train = Critic()

for i in range(epoch):
    for idx, x in enumerate(loader_critic):
        loss = ctr_train.train((x[0], x[1])) # Send the tuple directly 
        print(f"Epoch: {i}, Batch: {idx}, Loss: {loss.item()}")
        #* Critic's error gets very very low. Either its an overfit or the chosen model is really good

##Save model
# torch.save(ctr.model.state_dict(), "az_crt.pth")

CriticLSTM(
  (embedding): Embedding(382, 10)
  (lstm): LSTM(10, 64, num_layers=2, batch_first=True, dropout=0.3)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_1): Linear(in_features=50, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 0, Batch: 0, Loss: 0.6940577030181885
Epoch: 0, Batch: 1, Loss: 0.6966054439544678
Epoch: 0, Batch: 2, Loss: 0.6969747543334961
Epoch: 0, Batch: 3, Loss: 0.6934054493904114
Epoch: 0, Batch: 4, Loss: 0.6910766363143921
Epoch: 0, Batch: 5, Loss: 0.6902005076408386
Epoch: 0, Batch: 6, Loss: 0.6900182962417603
Epoch: 0, Batch: 7, Loss: 0.6907711625099182
Epoch: 0, Batch: 8, Loss: 0.6863670349121094
Epoch: 0, Batch: 9, Loss: 0.6899600028991699
Epoch: 0, Batch: 10, Loss: 0.681754469871521
Epoch: 0, Batch: 11, Loss: 0.6729028820991516
Epoch: 0, Batch: 12, Loss: 0.6833070516586304
Epoch: 0, Batch: 13, Loss: 0.6652792096138
Epoch: 0, Ba

In [6]:
# Test trained Critic on test dataset
# crt2 = Critic(load_trained=True)
# crt2.model.to(device)
# crt2.model.eval() # Puts model in evaluation mode

# # Cycle through the batches in the test dataset
# for idx, x in enumerate(loader_critic_tester):
#     # Cycle through each sample one at a time
#     x_batch = x[0] # All the sequences
#     y_batch = x[1] # All the test labels
#     running_score = []
#     for idx, x_in in enumerate(x_batch):
#         score = crt2.score(x_in.cuda(0).long()) # Only the music sequence
#         print(f"This sample score: -- {score}, expected: {y_batch[idx]}")
#         running_score.append(score)
#     print(len(running_score))
#     print(f"Good music probability -- {np.mean(np.array(running_score))}")
    
#     break

### Task 3: Average score for Critic vs Composer

In [7]:
# Load models from internet and initialize to evaluation mode
cps = Composer(load_trained=True)
cps.model.eval()

crt = Critic(load_trained=True)
crt.model.to(device)
crt.model.eval() 

# Load models by downloading weights from internet
num_exp = 100
lss_scores = []

for i in range(num_exp):
    f_name = f"midi_test_" + str(i) + f".midi"
    pth = midi_sav_dir + "/" + f_name
    midi = cps.compose(n = 200) # Numpy array of midi notes

    midi_torch = torch.LongTensor(midi) # Input to the Critic LSTM
    # print(midi_torch.shape[0])
    score = crt.score(midi_torch)
    print(f"This music score: {score}")

    lss_scores.append(score)

    # Write this music
    midi = seq2piano(midi)
    midi.write(pth)
    print()

print()
print(f"Average composer vs critic experiment: {np.mean(np.array(lss_scores))}")

DEBUG:Starting new HTTPS connection (1): docs.google.com:443
DEBUG:https://docs.google.com:443 "GET /uc?export=download&id=1EMLxoMZ5CaX49yckSwdfE7AL7vhvVVnN HTTP/1.1" 303 0
DEBUG:Starting new HTTPS connection (1): doc-0o-4c-docs.googleusercontent.com:443
DEBUG:https://doc-0o-4c-docs.googleusercontent.com:443 "GET /docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i2f2u388dctssaidr6gtr1hkdp5l0a11/1697578800000/09578454364954716887/*/1EMLxoMZ5CaX49yckSwdfE7AL7vhvVVnN?e=download&uuid=10552aa3-abff-4d13-b882-5f50c6da0607 HTTP/1.1" 200 454502


Done.
Unzipping...Done.
ComposerLSTM(
  (embedding): Embedding(382, 10)
  (lstm): LSTM(10, 64, num_layers=2, batch_first=True, dropout=0.3)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_1): Linear(in_features=50, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=382, bias=True)
  (softmax): Softmax(dim=1)
)

DEBUG:Starting new HTTPS connection (1): docs.google.com:443
DEBUG:https://docs.google.com:443 "GET /uc?export=download&id=1d2t0v5qfzYEnuiuJsd9XbukSBQelO1uM HTTP/1.1" 303 0
DEBUG:Starting new HTTPS connection (1): doc-08-4c-docs.googleusercontent.com:443
DEBUG:https://doc-08-4c-docs.googleusercontent.com:443 "GET /docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/a486cm5ki1lcr41d5bdb86hn90smfai0/1697578800000/09578454364954716887/*/1d2t0v5qfzYEnuiuJsd9XbukSBQelO1uM?e=download&uuid=9b8ea1cd-8bb3-4eae-9b44-d876cd08a4f7 HTTP/1.1" 200 257958


Done.
Unzipping...Done.
CriticLSTM(
  (embedding): Embedding(382, 10)
  (lstm): LSTM(10, 64, num_layers=2, batch_first=True, dropout=0.3)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_1): Linear(in_features=50, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Number of notes in Composer vocabulary: 382
Sequence length is completely divisible by 50


DEBUG:up without down for pitch 80 at time 4
DEBUG:consecutive downs for pitch 49 at time 2 and 4
DEBUG:up without down for pitch 62 at time 4
DEBUG:up without down for pitch 45 at time 4
DEBUG:up without down for pitch 80 at time 4
DEBUG:up without down for pitch 66 at time 4
DEBUG:up without down for pitch 79 at time 4
DEBUG:up without down for pitch 90 at time 4
DEBUG:up without down for pitch 50 at time 4
DEBUG:up without down for pitch 75 at time 4
DEBUG:up without down for pitch 75 at time 4
DEBUG:up without down for pitch 44 at time 4
DEBUG:up without down for pitch 92 at time 4
DEBUG:up without down for pitch 92 at time 4
DEBUG:up without down for pitch 62 at time 4
DEBUG:up without down for pitch 92 at time 4
DEBUG:up without down for pitch 43 at time 4
DEBUG:consecutive downs for pitch 51 at time 4 and 5
DEBUG:up without down for pitch 92 at time 5
DEBUG:up without down for pitch 49 at time 5
DEBUG:consecutive downs for pitch 51 at time 4 and 5
DEBUG:up without down for pitch

...

This music score: 0.75

Number of notes in Composer vocabulary: 382
Sequence length is completely divisible by 50
This music score: 0.9941290467977524

Number of notes in Composer vocabulary: 382
Sequence length is completely divisible by 50


DEBUG:consecutive downs for pitch 122 at time 2 and 2
DEBUG:up without down for pitch 122 at time 2
DEBUG:up without down for pitch 75 at time 3
DEBUG:up without down for pitch 66 at time 3
DEBUG:up without down for pitch 44 at time 3
DEBUG:up without down for pitch 72 at time 3
DEBUG:up without down for pitch 62 at time 3
DEBUG:up without down for pitch 52 at time 3
DEBUG:consecutive downs for pitch 65 at time 3 and 3
DEBUG:up without down for pitch 47 at time 3
DEBUG:up without down for pitch 50 at time 3
DEBUG:up without down for pitch 78 at time 3
DEBUG:up without down for pitch 62 at time 4
DEBUG:up without down for pitch 90 at time 4
DEBUG:up without down for pitch 50 at time 4
DEBUG:up without down for pitch 44 at time 4
DEBUG:up without down for pitch 80 at time 5
DEBUG:up without down for pitch 58 at time 5
DEBUG:up without down for pitch 87 at time 5
DEBUG:up without down for pitch 87 at time 5
DEBUG:consecutive downs for pitch 45 at time 3 and 5
DEBUG:up without down for pit

This music score: 0.75

Number of notes in Composer vocabulary: 382
Sequence length is completely divisible by 50
This music score: 1.0

Number of notes in Composer vocabulary: 382
Sequence length is completely divisible by 50


DEBUG:up without down for pitch 85 at time 2
DEBUG:up without down for pitch 31 at time 2
DEBUG:up without down for pitch 65 at time 2
DEBUG:up without down for pitch 53 at time 2
DEBUG:up without down for pitch 73 at time 2
DEBUG:up without down for pitch 99 at time 2
DEBUG:up without down for pitch 49 at time 2
DEBUG:up without down for pitch 60 at time 3
DEBUG:consecutive downs for pitch 86 at time 2 and 3
DEBUG:up without down for pitch 50 at time 3
DEBUG:up without down for pitch 90 at time 3
DEBUG:up without down for pitch 79 at time 3
DEBUG:up without down for pitch 87 at time 4
DEBUG:up without down for pitch 83 at time 4
DEBUG:up without down for pitch 71 at time 5
DEBUG:consecutive downs for pitch 61 at time 4 and 5
DEBUG:up without down for pitch 66 at time 5
DEBUG:up without down for pitch 44 at time 5
DEBUG:up without down for pitch 85 at time 5
DEBUG:up without down for pitch 57 at time 5
DEBUG:consecutive downs for pitch 46 at time 5 and 6
DEBUG:consecutive downs for pit

This music score: 0.75

Number of notes in Composer vocabulary: 382
Sequence length is completely divisible by 50
This music score: 0.9999996721744537


Average composer vs critic experiment: 0.8545009784400464
